In [ ]:
! pip install sentencepiece

In [ ]:
import random
random.seed(40)

In [ ]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer as tkz
from transformers import XLMRobertaForMultipleChoice as mpc
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/Datasets/copa.all.jsonl', 'r') as f:
    copa_data = [json.loads(line) for line in f]

# Shuffle the dataset
random.shuffle(copa_data)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(copa_data))
valid_size = len(copa_data) - train_size

# Split the dataset
train_data = copa_data[:train_size]
validation_data = copa_data[train_size:]

model_base = 'xlm-roberta-base'

In [ ]:
class COPADataset(Dataset):
    def __init__(self, copa_data, tokenizer):
        self.copa_data = copa_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.copa_data)

    def __getitem__(self, idx):
        data = self.copa_data[idx]
        premise = data['premise']
        hypotheses = [data['hypothesis1'], data['hypothesis2']]
        label = data['label']

        # Tokenize
        encoded_input = self.tokenizer([premise] * 2, hypotheses, padding='max_length', truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor(label)

        return encoded_input

In [ ]:
# Load tokenizer and model
tokenizer = tkz.from_pretrained(model_base)
model = mpc.from_pretrained(model_base)

# Prepare the DataLoader
copa_dataset = COPADataset(train_data, tokenizer)
dataloader = DataLoader(copa_dataset, batch_size=8, shuffle=True)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

print('Training done')

Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.719723105430603
Epoch: 0, Loss: 0.6929642558097839
Epoch: 0, Loss: 0.6991888284683228
Epoch: 0, Loss: 0.6818623542785645
Epoch: 0, Loss: 0.7025121450424194
Epoch: 0, Loss: 0.6790699362754822
Epoch: 0, Loss: 0.7133760452270508
Epoch: 0, Loss: 0.7157509922981262
Epoch: 0, Loss: 0.7127509713172913
Epoch: 0, Loss: 0.6504248380661011
Epoch: 0, Loss: 0.7164402604103088
Epoch: 0, Loss: 0.7112594842910767
Epoch: 0, Loss: 0.6887534856796265
Epoch: 0, Loss: 0.7411404252052307
Epoch: 0, Loss: 0.701660692691803
Epoch: 0, Loss: 0.7205138206481934
Epoch: 0, Loss: 0.7024257183074951
Epoch: 0, Loss: 0.7263143658638
Epoch: 0, Loss: 0.7006480693817139
Epoch: 0, Loss: 0.713907778263092
Epoch: 0, Loss: 0.6855623722076416
Epoch: 0, Loss: 0.7119945287704468
Epoch: 0, Loss: 0.6755951046943665
Epoch: 0, Loss: 0.6867516040802002
Epoch: 0, Loss: 0.6957339644432068
Epoch: 0, Loss: 0.6666485667228699
Epoch: 0, Loss: 0.6884198188781738
Epoch: 0, Loss: 0.6905801296234131
Epoch: 0, Loss: 0.67293947

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
run = 3
model_path = '/content/drive/MyDrive/Models/copa_' + model_base + '_' + str(run) + '.pt'
torch.save(model.state_dict(), model_path)

In [ ]:
model = mpc.from_pretrained(model_base)
model.load_state_dict(torch.load(model_path))

In [ ]:
tokenizer = tkz.from_pretrained(model_base)
validation_dataset = COPADataset(validation_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

accuracy, f1, precision, recall = [], [], [], []
batches = 0

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return [accuracy, f1, precision, recall]

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']  # ground truth

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)  # predictions

        calculated_metrics = compute_metrics(labels, predictions)
        accuracy.append(calculated_metrics[0])
        f1.append(calculated_metrics[1])
        precision.append(calculated_metrics[2])
        recall.append(calculated_metrics[3])
        batches += 1

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

print(f"Accuracy: {sum(accuracy)/batches:.4f}")
print(f"F1: {sum(f1)/batches:.4f}")
print(f"Precision: {sum(precision)/batches:.4f}")
print(f"Recall: {sum(recall)/batches:.4f}")

# Neuro Symbolic test

In [ ]:
class COPADataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        try:
            # Extracting data from the dataframe
            data = self.dataframe.iloc[idx]
            premise = data['premise']
            hypothesis1 = data['hypothesis1']
            hypothesis2 = data['hypothesis2']
            label = data['label']

            # Concatenating the premise and hypotheses with their respective relations
            premise_with_relations = premise + " " + data['premise_relations']
            hypothesis1_with_relations = hypothesis1 + " " + data['hypothesis1_relations']
            hypothesis2_with_relations = hypothesis2 + " " + data['hypothesis2_relations']

            # Tokenize
            encoded_input = self.tokenizer(
                [premise_with_relations] * 2,
                [hypothesis1_with_relations, hypothesis2_with_relations],
                padding='max_length',
                truncation=True,
                max_length=512,  # Or another suitable max length
                return_tensors='pt'
            )
            encoded_input['labels'] = torch.tensor(label)

            return encoded_input

        except KeyError as e:
            print(f"KeyError encountered at index {idx}: {e}")
            raise


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForMultipleChoice
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

processed_data = pd.read_csv('/content/processed_train_data.csv')
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

# Prepare the DataLoader
# Convert your dataframe to the updated dataset
train_dataset = COPADataset(processed_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not 

Epoch: 0, Loss: 0.7224762439727783


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch: 0, Loss: 0.6928279995918274


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

Epoch: 2, Loss: 0.6824203729629517


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch: 2, Loss: 0.6396059989929199


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch: 2, Loss: 0.7171453833580017


In [ ]:
val_data = pd.read_csv('/content/processed_validation_data.csv')
# Load tokenizer and model

validation_dataset = COPADataset(val_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)


In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Overall Accuracy: 0.4880
